In [1]:
# vocabulary : transfor input to number data
# vectorizer: set data as vector
# Dataset: process data's vectorizer
# Model: torch model

In [6]:
import os
from argparse import Namespace
import collections
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import torch
import pprint

In [4]:
def set_seeds(seed,cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda :
        torch.cuda.manual_seed_all(seed)

In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [7]:
args = Namespace(
    seed = 999,
    cuda = False,
    shuffle = True,
    data_file = "names.csv",
    split_data_file = "split_names.csv",
    vectorizer_file = "vectorizer.json",
    model_state_file = "model.pth",
    save_dir = "names",
    train_size = 0.7,
    val_size = 0.15,
    test_size = 0.15,
    num_epochs = 20,
    early_stopping_criteria = 5,
    learning_rate = 1e-3,
    batch_size = 64,
    hidden_dim = 300,
    dropout_p = 0.1
)

print(" args is :")
pprint.pprint(vars(args),indent=4)

 args is :
{   'batch_size': 64,
    'cuda': False,
    'data_file': 'names.csv',
    'dropout_p': 0.1,
    'early_stopping_criteria': 5,
    'hidden_dim': 300,
    'learning_rate': 0.001,
    'model_state_file': 'model.pth',
    'num_epochs': 20,
    'save_dir': 'names',
    'seed': 999,
    'shuffle': True,
    'split_data_file': 'split_names.csv',
    'test_size': 0.15,
    'train_size': 0.7,
    'val_size': 0.15,
    'vectorizer_file': 'vectorizer.json'}


In [ ]:
set_seeds(args.seed,args.cuda)
create_dir(args.save_dir)